In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
#importing the file that Guillaume sent us (data1)
with open ('./Hololens_data/p1.json') as json_file:
    data = json.load(json_file)

In [3]:
def norm(vect):
    sum = 0
    
    for el in vect:
        sum += el**2
    
    return np.sqrt(sum)

Let's create a Panda's dataFrame with position, time, rotatio, BPM  of each frame of the game and a second Dataframe with the balloons gathering data

In [51]:
def create_df_hand(game_file):
    with open(game_file) as json_file:
        data = json.load(json_file)
        
    df_game = pd.DataFrame(data['datasList'][0]['listLevelDatas'][0]['userDatas'])
    for i in range(1,len(data['datasList'][0]['listLevelDatas'])):
        df_game = pd.concat([df_game, pd.DataFrame(data['datasList'][0]['listLevelDatas'][i]['userDatas'])])
        
    
    
    #getting rid of the timeStamp's zero
    df_game = df_game[df_game['timeStamp']>0]
    
    #reset index after having got rid of the timeStamp zeros
    df_game = df_game.reset_index(drop = True) 
    
    #let's create three new columns, each one with one coordinate for df_game:
    #If they show later to be useless, we supprime these lines to get rid of them
    position =  df_game['headPos'].apply(pd.Series)
    df_game = pd.concat([df_game, position], axis=1)
    
    #Drops the duplicated rows in the Hololens DataSet
    indexes_to_drop = []
    for index, row in df_game.iterrows():
        if index != 0:
            if df_game.loc[index, 'timeStamp'] == df_game.loc[index-1, 'timeStamp']:
                indexes_to_drop.append(index)
    #print('length indexes_to_drop:', len(indexes_to_drop))
    df_game.drop(df_game.index[indexes_to_drop], inplace=True)
    df_game = df_game.reset_index(drop = True)
    

    #Fixes the bug in the timeOfDestroy and timeOfSpawn that came with Hololens Data (values were reseting)
    for index, row in df_game.iterrows():
        if index != 0:
            if df_game.loc[index, 'timeStamp'] < df_game.loc[index-1, 'timeStamp']:
                for idx in range(index,len(df_game)):
                    df_game.at[idx, 'timeStamp'] = df_game.at[idx, 'timeStamp']  +df_game.at[index-1, 'timeStamp']
                    
    #Here we create a column withe a 4-element tuple: (x,y,z,t) for each dataframe
    df_game['head_positions'] = df_game[['x', 'y', 'z', 'timeStamp', 'headRotationY']].apply(lambda x: tuple(x), axis=1)
                    
    
    
    return df_game

In [52]:
def create_df_balloon(game_file):
        
    with open(game_file) as json_file:
        data = json.load(json_file)
        
        
    df_balloon = pd.DataFrame(data['datasList'][0]['listLevelDatas'][0]['listBalloonDatas'])
    for i in range(1,len(data['datasList'][0]['listLevelDatas'])):
        df_balloon = pd.concat([df_balloon, pd.DataFrame(data['datasList'][0]['listLevelDatas'][i]['listBalloonDatas'])])
        
    df_balloon = df_balloon.reset_index(drop = True)
    
    #Drops the duplicated rows in the Hololens DataSet
    indexes_to_drop = []
    for index, row in df_balloon.iterrows():
        if index != 0:
            if df_balloon.loc[index, 'timeOfSpawn'] == df_balloon.loc[index-1, 'timeOfSpawn']:
                indexes_to_drop.append(index)
    df_balloon.drop(df_balloon.index[indexes_to_drop], inplace=True)
    
    df_balloon = df_balloon.reset_index(drop = True)
    
    #this part of the code fixes the bug in the timeOfDestroy and timeOfSpawn that came with Hololens Data (values were reseting)
    for index, row in df_balloon.iterrows():
        if index != 0:
            if df_balloon.loc[index, 'timeOfDestroy'] < df_balloon.loc[index-1, 'timeOfDestroy']:
                for idx in range(index,len(df_balloon)):
                    df_balloon.at[idx, 'timeOfDestroy'] = df_balloon.at[idx, 'timeOfDestroy']  + df_balloon.at[index-1, 'timeOfDestroy']
                    df_balloon.at[idx, 'timeOfSpawn'] = df_balloon.at[idx, 'timeOfSpawn'] + df_balloon.at[index-1, 'timeOfSpawn']
    
    return df_balloon

In [53]:
df_balloon = create_df_balloon('./Hololens_data/p8.json')

In [54]:
df_game = create_df_hand('./Hololens_data/p8.json')

In [55]:
df_balloon

,balloonInitialPosition,balloonPointGain,balloonTimout,balloonWasDestroyByUser,distance,lifeTime,timeOfDestroy,timeOfSpawn
0,"{'x': -0.2678895592689514, 'y': -0.39798125624...",20.0,False,True,1.051109,3.897904,3.931597,0.033694
1,"{'x': 0.29252728819847107, 'y': -0.30750298500...",25.0,False,True,0.490467,2.028822,6.079425,4.050603
2,"{'x': -0.49933886528015137, 'y': -0.2292173951...",30.0,False,True,0.173705,1.099373,9.210104,8.110731
3,"{'x': -0.5458201766014099, 'y': -0.21817447245...",5.0,False,True,1.500692,7.193855,17.320154,10.126299
4,"{'x': -0.7033659219741821, 'y': 0.100090458989...",20.0,False,True,1.500420,3.912960,22.166029,18.253069
5,"{'x': -0.6033926010131836, 'y': -0.24076703190...",15.0,False,True,1.041862,4.364701,26.563866,18.286205
6,"{'x': 0.36678409576416016, 'y': -0.16069257259...",0.0,False,False,0.283927,1.198722,28.962751,23.851069
7,"{'x': 0.11513650417327881, 'y': 0.017482461407...",5.0,False,True,4.266220,14.538195,43.832098,25.380944
8,"{'x': 1.029270887374878, 'y': -0.1800512075424...",25.0,False,True,0.634269,2.663246,47.695238,41.119032
9,"{'x': 0.819279134273529, 'y': -0.4253667294979...",0.0,False,False,0.037545,0.532780,49.127647,44.681908


In [56]:
df_game

,BPM,headPos,headRotationY,timeStamp,x,y,z,head_positions
0,1.0,"{'x': -0.11260681599378586, 'y': 0.01763401925...",235.545746,0.208880,-0.112607,0.017634,0.234992,"(-0.11260681599378586, 0.017634019255638123, 0..."
1,1.0,"{'x': -0.2026032954454422, 'y': 0.014482761733...",233.700165,0.302172,-0.202603,0.014483,0.261551,"(-0.2026032954454422, 0.014482761733233929, 0...."
2,1.0,"{'x': -0.22910091280937195, 'y': 0.01446275692...",231.035782,0.368441,-0.229101,0.014463,0.266605,"(-0.22910091280937195, 0.014462756924331188, 0..."
3,1.0,"{'x': -0.2467835247516632, 'y': 0.019006408751...",230.016541,0.434228,-0.246784,0.019006,0.269684,"(-0.2467835247516632, 0.019006408751010895, 0...."
4,1.0,"{'x': -0.2646417021751404, 'y': 0.018276344984...",230.751556,0.500752,-0.264642,0.018276,0.275719,"(-0.2646417021751404, 0.01827634498476982, 0.2..."
5,1.0,"{'x': -0.28640216588974, 'y': 0.01471910160034...",232.129791,0.567657,-0.286402,0.014719,0.284466,"(-0.28640216588974, 0.01471910160034895, 0.284..."
6,1.0,"{'x': -0.3113946318626404, 'y': 0.011165476404...",233.620285,0.645727,-0.311395,0.011165,0.292630,"(-0.3113946318626404, 0.011165476404130459, 0...."
7,1.0,"{'x': -0.3420345187187195, 'y': 0.011811528354...",234.465057,0.717524,-0.342035,0.011812,0.299909,"(-0.3420345187187195, 0.011811528354883194, 0...."
8,1.0,"{'x': -0.3615450859069824, 'y': 0.015759464353...",234.285187,0.784144,-0.361545,0.015759,0.302861,"(-0.3615450859069824, 0.015759464353322983, 0...."
9,1.0,"{'x': -0.37414634227752686, 'y': 0.01661998033...",233.474625,0.850483,-0.374146,0.016620,0.304063,"(-0.37414634227752686, 0.016619980335235596, 0..."


* The function `hand_positions` extracts the positions of the right hand along with the time corresponding to those positions. It returns an array of shape [(x, y, z, t)] (length number_of_position, with 4 elements arrays representing (x, y, z, t)).

In [57]:
def hand_positions(game_file):
    return list(create_df_hand(game_file)['head_positions'])
    

In [58]:
traj = hand_positions('./Hololens_data/p1.json')
#orientation(traj[0][0], traj[1][0], traj[0][1], traj[1][1], traj[0][2], traj[1][2])

In [60]:
hand_positions('./Hololens_data/p1.json')

[(-0.017746279016137123,
  0.07356427609920502,
  0.07732413709163666,
  0.12843038141727448,
  354.9891052246094),
 (-0.024730190634727478,
  0.07427263259887695,
  0.06569796800613403,
  0.5337522029876709,
  354.58966064453125),
 (-0.025495097041130066,
  0.07335755974054337,
  0.06575645506381989,
  0.5985556840896606,
  354.5259094238281),
 (-0.026182709261775017,
  0.07278983294963837,
  0.0659356638789177,
  0.6652300357818604,
  354.4717102050781),
 (-0.026342004537582397,
  0.07212531566619873,
  0.06530565023422241,
  0.7310526967048645,
  354.26873779296875),
 (-0.025485200807452202,
  0.07237708568572998,
  0.06402507424354553,
  0.7975941300392151,
  354.7206115722656),
 (-0.025491196662187576,
  0.07315880060195923,
  0.06265338510274887,
  0.8655750751495361,
  354.6669616699219),
 (-0.0263128113001585,
  0.07344095408916473,
  0.06170172989368439,
  0.9314557909965515,
  354.4627685546875),
 (-0.026026148349046707,
  0.07336080074310303,
  0.06112295389175415,
  0.99902

* The function `bubble_pop` extracts the time of each game event corresponding to the pop of a bubble by the player. It returns an array of shape [t] (length number_of_bubble_poped).

In [61]:
def bubble_pop(game_file):
    return list(create_df_balloon(game_file)['timeOfDestroy'])


In [62]:
bubble_pop('./Hololens_data/p1.json')

[7.609264850616455,
 13.753533363342285,
 19.449386596679688,
 26.677921295166016,
 29.54290771484375,
 33.43760681152344]

# Extraction of sub-trajectories & features
The function `sub_trajectories` returns an array of shape [[*[(x,y,t),(x,y,t),...]*, for each bubble in wave], for each wave]. To access all positions and time of the trajectory between the *i* and *i+1* bubble of the *n* wave : *sub_trajectories[n-1][i]*.

In [63]:
def sub_trajectories(game_file):
    hand_position = hand_positions(game_file)
    #bubble_pop_time = bubble_pop_clean(game_file)
    bubble_pop_time = bubble_pop(game_file)
    
    th = hand_position[0][3] #change to 3 because we have one more dimension
    
    sub_traj=[]
    
    nb_waves = len(bubble_pop_time)//5
    i=0 #loop count for waves
    k=0 #loop count for hand positions
    while i<nb_waves :
        sub_traj.append([])
        j=0 #loop count for bubbles
        while j<5:
            sub_traj[i].append([])
            t = bubble_pop_time[j+5*i] #the time the bubble was gathered
            while th < t:
                sub_traj[i][j].append(hand_position[k]) #appends the position of the hand and the corresponding time
                k+=1
                th = hand_position[k][3]
            j+=1
        i+=1
    
    return np.array(sub_traj)

In [67]:
print(sub_trajectories('./Hololens_data/p1.json')[0][0][1])

(-0.024730190634727478, 0.07427263259887695, 0.06569796800613403, 0.5337522029876709, 354.58966064453125)


We define some functions to extract interesting features from trajectories. We first look for Static features : 
* `length` returns the length of the trajectory *traj*
* `barycenter` returns the barycenter of the trajectory *traj* in shape (x,y)
* `location` returns the average distance of each point to the barycenter of the trajectory *traj*
* `location_max` returns the maximum distance between a point of the trajectory and the barycenter of this trajectory
* `orientation` returns the angle between points the line between *(x1, y1)* and *(x2, y2)* and the horizontal axis (in degrees)
* `orientation_feat` returns the preceeding feature for the first two points and the last two points of the trajectory *traj*
* `nb_turns` returns the number of turns in the trajectory *traj*, where a turn is detected if the orientation between two consecutive couples of points varies of more than *limit_angle*

In [21]:
def length(traj):
    l = 0
    
    for i in range(len(traj)-1):
        #l += np.sqrt((traj[i+1][0]-traj[i][0])**2 + (traj[i+1][1]-traj[i][1])**2) 
        l += np.sqrt((traj[i+1][0]-traj[i][0])**2 + (traj[i+1][1]-traj[i][1])**2+(traj[i+1][2]-traj[i][2])**2)
    
    return l

def barycenter(traj):
    x = 0
    y = 0
    z = 0
    n = len(traj)
    
    for i in range(n):
        x += traj[i][0]
        y += traj[i][1]
        z += traj[i][2]
    
    if n>0:
        return (x/n, y/n, z/n) #(x/n, y/n, z/n)
    else:
        return (0,0,0) #(0,0,0)

def location(traj):
    loc_avg = 0
    n = len(traj)
    p = barycenter(traj)
    
    for i in range(n):
        #loc_avg += np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2) 
        loc_avg += np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2+(traj[i][2]-p[2])**2)
        
    return loc_avg/n

def location_max(traj):
    n = len(traj)
    p = barycenter(traj)
    if n>0:
        l_max = np.max([np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2+(traj[i][2]-p[2])**2) for i in range(n)])
        #l_max = np.max([np.sqrt((traj[i][0] - p[0])**2 + (traj[i][1] - p[1])**2) for i in range(n)]) 
        return l_max
    else:
        return 0

    
def orientation(x1, x2 , y1, y2, z1, z2):
    #if x2 == x1 and y2>=y1:
    #    return 90
    #elif x2 == x1 and y2<=y1:
    #    return -90
    #else:
    #    return np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi) #in degree
    if x2-x1<0:
        return [np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi),np.arctan((z2 - z1)/(x2 - x1)) * (180/np.pi)+180] #in degree
    elif z2-z1<0 and x2-x1>0:
        return [np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi),np.arctan((z2 - z1)/(x2 - x1)) * (180/np.pi)] #in degree
    if x2 == x1 and y2>=y1 and z2==z1:
        return [90,0]
    elif x2 == x1 and y2<=y1 and z2==z1:
        return [-90,0]
    elif x2-x1>0 and z2-z1>=0:
        return [np.arctan((y2 - y1)/(x2 - x1)) * (180/np.pi),np.arctan((z2 - z1)/(x2 - x1)) * (180/np.pi)+180] #in degree

def orientation_feat(traj):
    n = len(traj)
    if n>1:
        #ts = orientation(traj[0][0], traj[1][0], traj[0][1], traj[1][1])
        #te = orientation(traj[-2][0], traj[-1][0], traj[-2][1], traj[-1][1])
        ts = orientation(traj[0][0], traj[1][0], traj[0][1], traj[1][1], traj[0][2], traj[1][2])
        te = orientation(traj[-2][0], traj[-1][0], traj[-2][1], traj[-1][1], traj[-2][2], traj[-1][2])

        return (ts, te)
    else:
        #return (0,0)
        return ([0,0],[0,0])

def nb_turns(traj, limit_angle):
    nb_turns = 0
    n=len(traj)
    
    for i in range(n-2):
        if(np.abs(orientation(traj[i][0], traj[i+1][0], traj[i][1], traj[i+1][1], traj[i][2], traj[i+1][2])[0] - orientation(traj[i+1][0], traj[i+2][0], traj[i+1][1], traj[i+2][1], traj[i+1][2], traj[i+2][2])[0]) > limit_angle):
            nb_turns += 1
        #if(np.abs(orientation(traj[i][0], traj[i+1][0], traj[i][1], traj[i+1][1]) - orientation(traj[i+1][0], traj[i+2][0], traj[i+1][1], traj[i+2][1])) > limit_angle):
        #    nb_turns += 1
    
    return nb_turns

We then define dynamic features:
* `velocity` returns the list of the point to point velocities over the whole trajectory *traj*
* `velocity_avg` returns the average velocity over the trajectory *traj*
* `velocity_max` returns the greatest velocity over the trajectory *traj*
* `velocity_min` returns the lowest velocity over the trajectory *traj*
* `nb_vmin` returns the number of local minimum of velocity
* `nb_vmax` returns the number of local maximum of velocity

In [84]:
np.abs(-5)

5

In [109]:
def velocity(traj):
    velocity = []
    
    for i in range(len(traj) - 1):
        #v = norm(np.array(traj)[i+1][:2] - np.array(traj)[i][:2]) / (np.array(traj)[i+1][3] - np.array(traj)[i][3])
        v = norm(np.array(traj)[i+1][:3] - np.array(traj)[i][:3]) / (np.array(traj)[i+1][3] - np.array(traj)[i][3])
        velocity.append(v)
        
    return np.array(velocity)
'''
def angular_speed(traj):
    angular_speed = []
    for i in range(len(traj) - 1):
        w = (np.array(traj)[i+1][4] - np.array(traj)[i][4]) / (np.array(traj)[i+1][3] - np.array(traj)[i][3])
        angular_speed.append(w)
        
    return np.array(angular_speed)
'''
def velocity_avg(traj):
    v_avg = 0
    n = len(traj)
    if n>1:
        v_list = velocity(traj)

        for i in range(n-1):
            v_avg += v_list[i]

        return v_avg/(n-1)
    else:
        return 0
'''
def angular_speed_avg(traj):
    w_avg = 0
    n = len(traj)
    if n>1:
        w_list = angular_speed(traj)

        for i in range(n-1):
            w_avg += w_list[i]

        return w_avg/(n-1)
    else:
        return 0 
'''
def velocity_max(traj):
    if len(traj)>1:
        return np.max(angular_speed(traj))
    else:
        return 0
'''
def angular_speed_max(traj):
    if len(traj)>1:
        return np.max(angular_speed(traj))
    else:
        return 0
'''
def velocity_min(traj):
    if len(traj)>1:
        return np.min(velocity(traj))
    else:
        return 0
'''
def angular_speed_min(traj):
    if len(traj)>1:
        return np.min(angular_speed(traj))
    else:
        return 0
'''
def nb_vmin(traj):
    nb = 0
    v_list = velocity(traj)
    
    for i in range(1,len(v_list)-1):
        if v_list[i]<v_list[i+1] and v_list[i]<v_list[i-1]:
            nb += 1
    
    return nb
'''
def nb_wmin(traj):
    nb = 0
    w_list = angular_speed(traj)
    
    for i in range(1,len(w_list)-1):
        if w_list[i]<w_list[i+1] and w_list[i]<w_list[i-1]:
            nb += 1
    
    return nb    
'''
def nb_vmax(traj):
    nb = 0
    v_list = velocity(traj)
    
    for i in range(1,len(v_list)-1):
        if v_list[i]>v_list[i+1] and v_list[i]>v_list[i-1]:
            nb += 1
    
    return nb

'''
def nb_wmax(traj):
    nb = 0
    w_list = angular_speed(traj)
    
    for i in range(1,len(w_list)-1):
        if w_list[i]>w_list[i+1] and w_list[i]>w_list[i-1]:
            nb += 1
    
    return nb
'''

'\ndef nb_wmax(traj):\n    nb = 0\n    w_list = angular_speed(traj)\n    \n    for i in range(1,len(w_list)-1):\n        if w_list[i]>w_list[i+1] and w_list[i]>w_list[i-1]:\n            nb += 1\n    \n    return nb\n'

The function `feature_vector` extracts features from the trajectory in argument *traj = [(x,y,z)]*

In [110]:
def bucketize_nb_turns(nb_turn):
    if nb_turn == 0:
        return [1, 0, 0, 0]
    elif nb_turn == 1:
        return [0, 1, 0, 0]
    elif nb_turn < 4: # 2 ou 3
        return [0, 0, 1, 0]
    else:
        return [0, 0, 0, 1] #4 ou plus

In [111]:
def bucketize_nb_v(nb_v):
    if nb_v < 2:
        return [1, 0, 0, 0]
    elif nb_v < 4: # 2 ou 3
        return [0, 1, 0, 0]
    elif nb_v < 6: # 4 ou 5
        return [0, 0, 1, 0]
    else:
        return [0, 0, 0, 1] # 6 ou plus

In [114]:
def feature_vector(traj, playerID, ballon, game_area, time, limit_angle=0.25, Listefeatures=["nb_wmax", "nb_wmin", "angular_speed_min","angular_speed_max", "angular_speed_avg", "angular_speed", "dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]):
    diag = np.sqrt(game_area[0]**2 + game_area[1]**2)
    if len(traj)==0:
        return []
    if time=='wave':
        listetot=[]
        for k in range(len(traj)):
            listetot+=traj[k]
        feature_vector = [playerID]
        dist=length(listetot)
        bc=barycenter(listetot)
        if "nb_wmax" in Listefeatures:
            feature_vector.append(nb_wmax(listetot))
        if "nb_wmin" in Listefeatures:
            feature_vector.append(nb_wmin(listetot))
        if "angular_speed_min" in Listefeatures:
            feature_vector.append(angular_speed_min(listetot))
        if "angular_speed_max" in Listefeatures:
            feature_vector.append(angular_speed_max(listetot))
        if "angular_speed_avg" in Listefeatures:
            feature_vector.append(angular_speed_avg(listetot))
        if "angular_speed" in Listefeatures:
            feature_vector.append(angular_speed(listetot))
        if "dist/diag" in Listefeatures:
            feature_vector.append(dist/diag)
        if "game area" in Listefeatures:
            feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
            feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
        if location_max(listetot) == 0 and "barycenter distance" in Listefeatures:
            feature_vector.append(np.float64(0))
        elif "barycenter distance" in Listefeatures:
            feature_vector.append(location(listetot)/location_max(listetot))
        angles = 0.5 + np.array(orientation_feat(listetot)) / 180 # between 0 and 1
        if "angles" in Listefeatures:
            feature_vector.append(angles[0][0]) #first orientation of traj
            feature_vector.append(angles[0][1])
            feature_vector.append(angles[1][1])#last orientation of traj
            feature_vector.append(angles[1][1])
        if "nb turns" in Listefeatures:
            feature_vector.append(nb_turns(listetot, limit_angle))
        if "velocity average" in Listefeatures:
            feature_vector.append(velocity_avg(listetot))
        if "velocity min" in Listefeatures:
            feature_vector.append(velocity_min(listetot))
        if "velocity max" in Listefeatures:
            feature_vector.append(velocity_max(listetot))
        if "number of mins" in Listefeatures:
            feature_vector.append(nb_vmin(listetot))
        if "number of maxs" in Listefeatures:
            feature_vector.append(nb_vmax(listetot))
        return feature_vector
    if time=='pop':
        listetot=[]
        for k in range(len(traj)):
            listetot+=traj[k]
        if len(listetot)==0:
            return []
        listetot2=[]
        sousliste=[]
        compteur=0
        for k in range(len(listetot)):
            if k==len(listetot)-1:
                listetot2.append(sousliste)
                compteur+=1
                sousliste=[]
            if listetot[k][3]<ballon[compteur]:
                sousliste.append(listetot[k])
            else:
                listetot2.append(sousliste)
                sousliste=[]
                compteur+=1
        print("nb de features:", len(listetot2))
        listefeatures=[]
        for element in listetot2:
            if len(element)!=0:
                feature_vector = [playerID]
                dist=length(element)
                bc=barycenter(element)
                if "dist/diag" in Listefeatures:
                    feature_vector.append(dist/diag)
                if "game area" in Listefeatures:
                    feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
                    feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
                if location_max(element) == 0 and "barycenter distance" in Listefeatures:
                    feature_vector.append(np.float64(0))
                elif "barycenter distance" in Listefeatures:
                    feature_vector.append(location(element)/location_max(element))
                angles = 0.5 + np.array(orientation_feat(element)) / 180 # between 0 and 1
                if "angles" in Listefeatures:
                    feature_vector.append(angles[0][0]) #first orientation of traj
                    feature_vector.append(angles[0][1])
                    feature_vector.append(angles[1][1])#last orientation of traj
                    feature_vector.append(angles[1][1])
                if "nb turns" in Listefeatures:
                    feature_vector.append(nb_turns(element, limit_angle))
                if "velocity average" in Listefeatures:
                    feature_vector.append(velocity_avg(element))
                if "velocity min" in Listefeatures:
                    feature_vector.append(velocity_min(element))
                if "velocity max" in Listefeatures:
                    feature_vector.append(velocity_max(element))
                if "number of mins" in Listefeatures:
                    feature_vector.append(nb_vmin(element))
                if "number of maxs" in Listefeatures:
                    feature_vector.append(nb_vmax(element))
                listefeatures.append(feature_vector)
        return listefeatures
    else:
        listetot=[]
        for k in range(len(traj)):
            listetot+=traj[k]
        if len(listetot)==0:
            return []
        tempslimit=time+listetot[0][3]
        listetot2=[]
        sousliste=[]
        for k in range(len(listetot)):
            if k==len(listetot)-1:
                listetot2.append(sousliste)
                tempslimit=listetot[k][3]+time
                sousliste=[]
            if listetot[k][3]<=tempslimit:
                sousliste.append(listetot[k])
            else:
                listetot2.append(sousliste)
                tempslimit=listetot[k][3]+time
                sousliste=[]
        print("nb de features:", len(listetot2))
        listefeatures=[]
        for element in listetot2:
            if len(element)!=0:
                feature_vector = [playerID]
                dist=length(element)
                bc=barycenter(element)
                if "dist/diag" in Listefeatures:
                    feature_vector.append(dist/diag)
                if "game area" in Listefeatures:
                    feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
                    feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
                if location_max(element) == 0 and "barycenter distance" in Listefeatures:
                    feature_vector.append(np.float64(0))
                elif "barycenter distance" in Listefeatures:
                    feature_vector.append(location(element)/location_max(element))
                angles = 0.5 + np.array(orientation_feat(element)) / 180 # between 0 and 1
                if "angles" in Listefeatures:
                    feature_vector.append(angles[0][0]) #first orientation of traj
                    feature_vector.append(angles[0][1])
                    feature_vector.append(angles[1][1])#last orientation of traj
                    feature_vector.append(angles[1][1])
                if "nb turns" in Listefeatures:
                    feature_vector.append(nb_turns(element, limit_angle))
                if "velocity average" in Listefeatures:
                    feature_vector.append(velocity_avg(element))
                if "velocity min" in Listefeatures:
                    feature_vector.append(velocity_min(element))
                if "velocity max" in Listefeatures:
                    feature_vector.append(velocity_max(element))
                if "number of mins" in Listefeatures:
                    feature_vector.append(nb_vmin(element))
                if "number of maxs" in Listefeatures:
                    feature_vector.append(nb_vmax(element))
                listefeatures.append(feature_vector)
        return listefeatures
    

In [115]:
def feature_vector_bucket(traj, playerID, game_area, limit_angle=0.25):
    diag = np.sqrt(game_area[0]**2 + game_area[1]**2)
    
    feature_vector = [playerID]
    
    feature_vector.append(length(traj)/diag)
    
    bc = barycenter(traj)
    feature_vector.append(np.float64(0.5 + bc[0] / game_area[0])) # between 0 and 1
    feature_vector.append(np.float64(0.5 + bc[1] / game_area[1]))
    
    if location_max(traj) == 0:
        feature_vector.append(np.float64(0))
    else:
        feature_vector.append(location(traj)/location_max(traj))
    
    angles = 0.5 + np.array(orientation_feat(traj)) / 180 # between 0 and 1
    feature_vector.append(angles[0]) #first orientation of traj
    feature_vector.append(angles[1]) #last orientation of traj
    
    bucket = bucketize_nb_turns(nb_turns(traj, limit_angle))
    for i in bucket:
        feature_vector.append(i)
    
    v_max = velocity_max(traj)
    if v_max == 0:
        feature_vector.append(0)
        feature_vector.append(0)
        feature_vector.append(0)
    else:
        feature_vector.append(velocity_avg(traj) / v_max)

        feature_vector.append(velocity_min(traj) / v_max)
        feature_vector.append(v_max)
    
    bucket_min = bucketize_nb_v(nb_vmin(traj))
    bucket_max = bucketize_nb_v(nb_vmax(traj))
    for i in bucket_min:
        feature_vector.append(i)
    for j in bucket_max:
        feature_vector.append(j)
    
    return feature_vector

The function `feature_vectors_game` allows to create the feature vectors over all the trajectories between the gathering of two bubbles of one game. The returned array is an array of multiple 13x5 arrays (the five feature vectors, containing 13 features each, corresponding to the five trajectories of each wave).

In [116]:
import ntpath

def feature_vectors_game(game_file, time, game_area = [21,10], limit_angle=0.25, Listefeatures=["nb_wmax", "nb_wmin", "angular_speed_min","angular_speed_max", "angular_speed_avg", "angular_speed", "dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]):
    balloon=list(create_df_balloon(game_file)['timeOfDestroy'])
    trajectories = sub_trajectories(game_file)
    nb_waves = len(trajectories)
    playerID = ntpath.basename(game_file)[:-4] #gets the name of the file as the player identity
    vectors = []
    for traj in trajectories:
        if time=='wave':
            vectors.append(feature_vector(traj, playerID, balloon, game_area,time,limit_angle=0.25, Listefeatures=["nb_wmax", "nb_wmin", "angular_speed_min","angular_speed_max", "angular_speed_avg", "angular_speed", "dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]))
        else:
            for feature_vec in feature_vector(traj, playerID, balloon, game_area,time,limit_angle=0.25, Listefeatures=["nb_wmax", "nb_wmin", "angular_speed_min","angular_speed_max", "angular_speed_avg", "angular_speed", "dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]):
                vectors.append(feature_vec)

    return np.array(vectors)

In [117]:
def simple_features_generator(game_list,time='wave', game_area = [21,10], limit_angle=0.25, Listefeatures=["nb_wmax", "nb_wmin", "angular_speed_min","angular_speed_max", "angular_speed_avg", "angular_speed", "dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]):
    features=[]
    labels=[]
    for file in game_list:
        for layer1 in feature_vectors_game(file,time,game_area = [21,10], limit_angle=0.25, Listefeatures=["nb_wmax", "nb_wmin", "angular_speed_min","angular_speed_max", "angular_speed_avg", "angular_speed", "dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"]):
            if len(layer1)==0:
                pass
            else:
                labels.append(layer1[0])
                features.append(layer1[1:])
    np.savetxt('features.csv', features, delimiter=",", fmt='%s')
    np.savetxt('output.csv', labels,delimiter=",", fmt='%s')
    return features, labels

The following functions provide different shapes for the feature vector. This way of creating the feature vector could be improved by using tensorflow and its feature vectors, instead of creating it "by hand".
* "concat" means all features are concatenated into one numpy vector for each sample
* "bucket" means it uses the bucketized version of the feature vector (for nb_turns, nb_vmin, nb_vmax)
* "hands"  means it uses the hand used to play as label instead of the player's ID

In [118]:
def feature_vectors_game_concat(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    playerID = 1 #int(parse_root(game_file)[2][0].text) ## CHANGE THIS PART #################################
    vectors = []
    
    for i in range(nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i] = vectors[i] + list(feature_vector(traj, playerID, game_area)[1:])
        vectors[i].append(playerID) ## CHANGE THIS PART #################################
    
    return np.array(vectors)

In [119]:
def feature_vectors_bucket_game_concat(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    playerID = 1#int(parse_root(game_file)[2][0].text)
    vectors = []
    
    for i in range(nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i] = vectors[i] + list(feature_vector_bucket(traj, playerID, game_area)[1:])
        vectors[i].append(playerID)
    
    return np.array(vectors)

In [120]:
def feature_vectors_bucket_game_concat_hands(game_file, game_area = [21,10]):
    trajectories = np.array(sub_trajectories(game_file))
    nb_waves = len(trajectories)
    if parse_root(game_file)[2][2].text == 'false':
        useRightHand = 0
    else:
        useRightHand = 1
    vectors = []
    
    for i in range(nb_waves):
        vectors.append([])
        for traj in trajectories[i]:
            vectors[i] = vectors[i] + list(feature_vector_bucket(traj, useRightHand, game_area)[1:])
        vectors[i].append(useRightHand)
    
    return np.array(vectors)

Finally we provide a function to get the agregation of all feature vectors over multiple game files, where *game_files* is the list of the names (String type) of all the game files to be considered.

In [121]:
def agregate_feature_vectors(game_files):
    vectors = []
    for file in game_files:
        vectors = vectors + list(feature_vectors_game_concat(file))
    
    return np.array(vectors)

In [122]:
def agregate_feature_vectors_bucket(game_files):
    vectors = []
    for file in game_files:
        vectors = vectors + list(feature_vectors_bucket_game_concat(file))
    
    return np.array(vectors)

In [123]:
def agregate_feature_vectors_bucket_hands(game_files):
    vectors = []
    for file in game_files:
        vectors = vectors + list(feature_vectors_bucket_game_concat_hands(file))
    
    return np.array(vectors)

# Export of the final data

In [124]:
def export_feature_vectors(vectors, name):
    np.savetxt(name, vectors, delimiter=",")

In [125]:
relative_path = 'C:/Users/menoci/Desktop/Studies/autisme et ML/Code+Data/xml_data/'
print(relative_path+'abc.xml')

C:/Users/menoci/Desktop/Studies/autisme et ML/Code+Data/xml_data/abc.xml


In [126]:
relative_path = './Hololens_data/'

game_files=[relative_path + 'p1.json',
           relative_path + 'p3.json',
           relative_path + 'p4.json',
           relative_path + 'p5.json',
           relative_path + 'p6.json',
           relative_path + 'p7.json',
           relative_path + 'p8.json',
           relative_path + 'p9.json',
           relative_path + 'p10.json'] #here we add all the files we want to train the algorithm with

#vect = agregate_feature_vectors_bucket(game_files)
#export_feature_vectors(vect, "kate_data.csv")

In [127]:
X, y = simple_features_generator(['./Hololens_data/p8.json'], 'wave', Listefeatures=["dist/diag","game area","barycenter distance","angles","nb turns","velocity average","velocity min","velocity max","number of mins","number of maxs"])

In [128]:
X

[array([67, 67, -5388.640257885855, 7157.505704379491, 4.634161451821374,
        array([-1.97829705e+01, -4.02051301e+01, -1.54932222e+01,  1.10487591e+01,
         2.05999268e+01,  1.90918229e+01,  1.17660330e+01, -2.69998076e+00,
        -1.22183731e+01, -1.36946991e+01, -2.70678905e+01, -2.76577323e+01,
        -1.50476556e+01,  1.20737933e+01,  3.59358834e+01,  3.73656646e+01,
         1.62672256e+01,  2.73306681e+01,  6.98575782e+01,  9.75751725e+01,
         7.82205590e+01,  1.05823923e+01,  7.73813823e+00,  5.76252556e+01,
         6.20696124e+01,  4.76122540e+01,  3.91123468e+01,  4.47952843e+01,
         7.09888326e+01,  5.57621052e+01,  3.85706470e+01,  3.94164137e+01,
         3.47508620e+01,  2.30101129e+01,  4.59441347e+01,  1.05615897e+02,
         1.11710973e+02,  7.45166345e+01,  9.16992401e+01,  7.50561579e+01,
         6.42967584e+01,  5.96100544e+01,  5.68964076e+01,  6.08451259e+01,
         4.07066379e+01,  1.63206734e+01, -4.24977312e+00, -1.09748471e+01,
       

In [129]:
pd.DataFrame(X)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,67,67,-5388.640258,7157.505704,4.634161,"[-19.78297053994135, -40.20513005010247, -15.4...",0.252593,0.490764,0.504158,0.261540,0.511141,1.408655,1.747410,1.747410,328,0.266006,0.002257,7157.505704,50,49
1,92,92,-7225.808605,5419.843111,-20.068777,"[-7225.8086054947025, 21.21334554496628, 21.48...",0.343020,0.501170,0.503811,0.677022,0.424138,1.724408,1.335612,1.335612,400,0.305821,0.023832,5419.843111,75,75
2,135,135,-5296.758581,5387.628621,7.383093,"[-85.15968920329352, -107.62773973583933, -65....",0.411173,0.502179,0.503376,0.430250,0.520003,1.297151,1.790313,1.790313,607,0.230505,0.007748,5387.628621,92,93
3,115,116,-5290.935817,5352.343478,-6.582093,"[6.148477670576001, -12.23666078241558, 14.139...",0.448925,0.536703,0.499255,0.459386,0.559818,1.675275,1.706286,1.706286,559,0.266506,0.007307,5352.343478,99,98
4,146,145,-5359.689792,5426.140713,-2.420884,"[-21.132735219691742, -10.971789329996588, -31...",0.442058,0.506560,0.501209,0.511431,0.375324,1.692008,1.721257,1.721257,640,0.215909,0.001955,5426.140713,113,113
